# Selección de Conjunto de Empresas para Entrenar y Validar modelo

##### Estudiantes:

- Obed Rios-Ruiz
- Nicolas Prieto-Escobar
- Pablo A. Saldarriaga-Aristizabal

Como parte de las tareas que se tienen en el desarrollo del trabajo, está la partición del conjunto de datos en dos subconjuntos, uno para la creación/entrenamiento del modelo, y otro conjunto de datos que permita evaluar la el desempeño o capacidad predictiva del modelo creado en un conjunto de datos que el modelo no ha visto. Para el trabajo, estamos considerando información de variables macroeconómicas entre los años 2016 a 2018, al igual que información de los costos de venta de 33 empresas del sector de la Construcción.

Por las características del conjunto de datos, al igual que del problema que se estudia, se considera conveniente realizar la partición de los datos en base a las empresas. Por lo tanto, se desea tener la información de 22 empresas para entrenar el modelo, y la información de las 33 restantes para probar el modelo.

In [1]:
### Realizamos la importación de los paquetes necesarios para la ejecución del notebook
import numpy as np
import pandas as pd
from itertools import combinations

In [2]:
### Realizamos la lectura del conjunto de datos, además de obtener una lista de todos los NITs de las empresas que
### estamos considerando
data = pd.read_csv('Datos_completos.csv')
data = data.drop(columns = ['Costo de ventas','Gastos de ventas'])
empresas = data['NIT'].unique()

In [3]:
### Cantidad de empresas
print(f'Se tienen {len(empresas)} empresas diferentes en el conjunto de datos')

Se tienen 33 empresas diferentes en el conjunto de datos


Dentro del proceso de evaluación de qué tan bueno es la combinación de empresas considerada en los conjuntos de train y test, consideraremos la métrica del error cuadrático entre el promedio de diferencia de costos de ventas para cada conjunto, por lo que la expresión corresponde a:

$$
EC =  (\dfrac{1}{22}\sum_{i=1}^{22} DataEmpresaTrain(CostoDeVentasDiff) -
\dfrac{1}{11}\sum_{i=1}^{11} DataEmpresaTest(CostoDeVentasDiff))^{2}
$$

Asi en el proceso que sigue a continuación, consideraremos las primeras 100.000 combinaciones posibles de las empresas para el conjunto train y test (obtenidos por la función combinations del paquete itertools), y nos quedaremos con aquella combinación de conjuntos que minimice el EC mencionado anteriormente

In [4]:
def diff_media(data, empresas1, empresas2):
    
    data_empresa1 = data[data['NIT'].isin(empresas1)]['Costo de ventas_dif'] 
    data_empresa2 = data[data['NIT'].isin(empresas2)]['Costo de ventas_dif']
  
    mean_empresa1 = data_empresa1.mean()
    mean_empresa2 = data_empresa2.mean()
    
    norm_dif = (mean_empresa1-mean_empresa2)**2
    
    return norm_dif

In [5]:
%%time

np.random.seed(42)
best_norm = np.inf
best_empresa_comb = None
lim = 100000
cont = 0
for p in combinations(empresas, 22):
    
    emp1 = [emp for emp in empresas if emp in p]
    emp2 = [emp for emp in empresas if not emp in emp1]
    
    norm_value = diff_media(data, emp1, emp2)
    
    if norm_value < best_norm:
        print(norm_value, cont)
        best_empresa_com = (emp1,emp2)
        best_norm = norm_value
    
    cont += 1
    if cont == lim:
        break

0.016770199391232855 0
0.0038988992173910626 2
0.0020920726060926495 34
8.724805312215589e-05 41
4.217734972101327e-05 251
3.270327613552945e-06 264
1.360494145224962e-06 661
5.324934345321799e-07 759
2.512490867977149e-07 760
1.5235191104070993e-08 1055
7.155460472918056e-09 4131
4.5359346093228465e-09 4232
2.611001864883461e-09 9344
2.3140959472729717e-10 19081
1.7335723581479037e-10 23527
5.297777131209481e-12 23930
3.2645956171741084e-12 99899
Wall time: 2min 6s


In [6]:
data_empresas1 = data[data['NIT'].isin(best_empresa_com[0])]['Costo de ventas_dif'] 
print(f'El promedio de la diferencia de los costos de venta en el conjunto de entrenamiento es {data_empresas1.mean()}')

El promedio de la diferencia de los costos de venta en el conjunto de entrenamiento es -0.03801533473757241


In [7]:
data_empresas2 = data[data['NIT'].isin(best_empresa_com[1])]['Costo de ventas_dif'] 
print(f'El promedio de la diferencia de los costos de venta en el conjunto de entrenamiento es {data_empresas2.mean()}')

El promedio de la diferencia de los costos de venta en el conjunto de entrenamiento es -0.038013527918373605


In [8]:
mean_empresas1 = data_empresas1.mean()
mean_empresas2 = data_empresas2.mean()

norm_dif = (mean_empresas1-mean_empresas2)**2

print(f'Validamos el valor del error cuadratico obtenido: {norm_dif}')

Validamos el valor del error cuadratico obtenido: 3.2645956171741084e-12


In [9]:
print(f"las empresas a considerar en el conjunto de entrenamiento son: {best_empresa_com[0]}")

las empresas a considerar en el conjunto de entrenamiento son: [800015615, 800045720, 800081030, 800112440, 800118660, 800157469, 800232356, 800236890, 801002644, 805012368, 806014553, 830030574, 830037495, 860009694, 860033653, 860050956, 890909034, 900173460, 900184722, 900204182, 900364670, 900378893]


In [10]:
print(f"las empresas a considerar en el conjunto de prueba son: {best_empresa_com[1]}")

las empresas a considerar en el conjunto de prueba son: [830052054, 860030360, 860501682, 890117431, 890300012, 890311366, 890904459, 890929951, 900234565, 900389088, 900437650]


Seguido este proceso, utilizamos los conjuntos de empresas obtenidos en este Notebook para realizar la partición del conjunto de datos en el archivo de Markdown que incluye todo el proceso de modelación y creación de modelos.